In [40]:
import re
import copy
import pickle
import requests
import datetime
import pandas as pd
import yfinance as yf
from tqdm.notebook import tqdm
from bs4 import BeautifulSoup
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
BASE_URL = 'https://www.fool.com'
EARNINGS_BASE_URL = 'https://www.fool.com/earnings-call-transcripts/?page='

In [3]:
# Return source code for every page in the range specified
def get_pages(pages):
    pbar = tqdm(total=pages, ncols=1000, bar_format='{l_bar}{bar}{n_fmt}/{total_fmt}')
    page_list = []
    for page_ in range(1, pages+1):
        pbar.update(1)
        pbar.set_description("Processing page %s" % page_)
        transcripts_response = requests.get(f'{EARNINGS_BASE_URL}{page_}')
        if transcripts_response.status_code==403:
            print(403)
            break
        transcripts_response_bs = BeautifulSoup(transcripts_response.content, 'html.parser')
        transcripts_ = transcripts_response_bs.find("div", {"class":"list-content", "id":f"page-{page_}"})
        page_list.append(transcripts_)
    return page_list

In [4]:
pages = get_pages(50)
print(f'{len(pages)} pages of transcripts')

50 pages of transcripts


In [5]:
# Returnd ticker
def get_ticker_title(heading_title):
    matched_ticker = re.findall('\(([A-Z]*)\)', heading_title)
    if len(matched_ticker) != 1:
        return None
    return matched_ticker[0]

In [6]:
def get_company_name(heading_title, ticker_):
    matched_company = re.findall(f'(.+)\s\({ticker_}', heading_title)
    if matched_company:
        return matched_company[0]
    else:
        return None

In [7]:
def get_fiscal_period(heading_):
    fiscal_period = re.findall('Q\d\s20\d{2}', heading_)
    if fiscal_period:
        return fiscal_period[0]
    else:
        return None

In [8]:
def get_period_ending(heading_):
    try:
        period_ = heading_.article.find("div", {'class':'text'}).p.get_text()
        period_ending = re.findall('period ending (.*)?', period_)
    except:
        period_ending = None
    return period_ending[0]

In [9]:
def get_transcript_date(heading_):
    try:
        footer_ = heading_.article.find("div", {'class':'text'}).div.get_text()
        transcript_date = footer_[footer_.index("|")+2:]
    except:
        transcript_date = None
    return transcript_date

In [10]:
def get_transcript_list(pages_source):
    pbar_page = tqdm(total=len(pages_source), ncols=1000, bar_format='{l_bar}{bar}{n_fmt}/{total_fmt}')
    transcript_list = []
    for page_num, page_source in enumerate(pages_source, 1):
        pbar_page.update(1)
        pbar_page.set_description("Processing page %s" % page_num)
        for heading_num, heading_ in enumerate(page_source.find_all('a')):
            heading_title = heading_["title"]
            ticker_ = get_ticker_title(heading_title)
            company_name = get_company_name(heading_title, ticker_)
            fiscal_period = get_fiscal_period(heading_title)
            period_ending = get_period_ending(heading_)
            transcript_date_ = get_transcript_date(heading_)
            heading_info = [ticker_, company_name, fiscal_period, period_ending, transcript_date_, heading_["href"]]
            transcript_list.append(heading_info)
    return transcript_list

In [11]:
transcript_list = get_transcript_list(pages)
print(f'{len(transcript_list)} transcripts')
transcript_list[0]

1000 transcripts


['NG',
 'Novagold Resources Inc',
 'Q2 2020',
 'May 31, 2020.',
 'Jun 25, 2020',
 '/earnings/call-transcripts/2020/06/25/novagold-resources-inc-ng-q2-2020-earnings-call-tr.aspx']

In [12]:
def get_transcript_sources(list_):
    pbar = tqdm(total=len(list_), ncols=1000, bar_format='{l_bar}{bar}{n_fmt}/{total_fmt}')
    transcripts_ = []
    for ticker_, transcript_link in list_:
        pbar.update(1)
        pbar.set_description("Processing company %s" % ticker_)
        transcript_response = requests.get(f'{BASE_URL}{transcript_link}')
        if transcript_response.status_code==403:
            print("403")
            break
        transcript_response_bs = BeautifulSoup(transcript_response.content, 'html.parser')
        article_ = transcript_response_bs.find("span", {"class":"article-content"})
        transcripts_.append(article_)
    return transcripts_

In [13]:
transcripts = get_transcript_sources([[transcript_[0], transcript_[5]] for transcript_ in transcript_list])
print(f'{len(transcripts)} transcripts processed')

1000 transcripts processed


In [278]:
def get_announcement_time(transcript_list, transcripts):
    upload_times = []
    new_upload_dates = []
    for transcript_info, transcript in zip(transcript_list, transcripts):
        all_text = transcript.get_text()
        upload_date = transcript_info[4]
        try:
            upload_time = re.findall(f'{upload_date}' + ',\s?([0-9]{1,2}:[0-9]{2}\s[a|p]\.m\.)', all_text)[0]
            new_upload_date = None
        except:
            try:
                upload_time = re.findall(',\s?([0-9]{1,2}:[0-9]{2}\s[a|p]\.m\.) ET', all_text)[0]
                new_upload_date = re.findall('([A-Z][a-z]{2}\.?\s[0-9]{1,2}?,\s20[1|2][0-9]),\s' + f'{upload_time}', all_text)[0]
            except:
                upload_time = None
                new_upload_date = None
        upload_times.append(upload_time)
        new_upload_dates.append(new_upload_date)
    return upload_times, new_upload_dates

In [287]:
#announcement_times, new_dates = get_announcement_time(transcript_list, transcripts)
#new_dates_transcript_list = copy.deepcopy(transcript_list)
#for each, new_date in zip(new_dates_transcript_list, new_dates):
#    if new_date:
#        each[4] = new_date
new_dates_transcript_list

[['NG',
  'Novagold Resources Inc',
  'Q2 2020',
  'May 31, 2020.',
  'Jun 25, 2020',
  '/earnings/call-transcripts/2020/06/25/novagold-resources-inc-ng-q2-2020-earnings-call-tr.aspx'],
 ['SNX',
  'SYNNEX Corporation',
  'Q2 2020',
  'May 31, 2020.',
  'Jun 25, 2020',
  '/earnings/call-transcripts/2020/06/25/synnex-corporation-snx-q2-2020-earnings-call-trans.aspx'],
 ['PRGS',
  'Progress Software',
  'Q2 2020',
  'March 31, 2020.',
  'Jun 25, 2020',
  '/earnings/call-transcripts/2020/06/25/progress-software-prgs-q2-2020-earnings-call-trans.aspx'],
 ['NKE',
  'Nike Inc',
  'Q4 2020',
  'May 31, 2020.',
  'Jun 25, 2020',
  '/earnings/call-transcripts/2020/06/25/nike-inc-nke-q4-2020-earnings-call-transcript.aspx'],
 ['CAMP',
  'CalAmp Corp',
  'Q1 2021',
  'May 31, 2020.',
  'Jun 25, 2020',
  '/earnings/call-transcripts/2020/06/25/calamp-corp-camp-q1-2021-earnings-call-transcript.aspx'],
 ['FUL',
  'H.B. Fuller',
  'Q2 2020',
  'March 31, 2020.',
  'Jun 25, 2020',
  '/earnings/call-transc

In [288]:
def get_transcript(transcript_list, transcripts):
    pbar = tqdm(total=len(transcript_list), ncols=1000, bar_format='{l_bar}{bar}{n_fmt}/{total_fmt}')
    corpuses = []
    for transcript_info, transcript in zip(transcript_list, transcripts):
        pbar.update(1)
        pbar.set_description("Extracting transcript %s" % transcript_info[0])
        first_speaker = transcript.find(string=' -- ')
        if not first_speaker:
            corpuses.append(None)
            continue
        entire_corpus = first_speaker.find_parent().find_next_siblings()
        corpuses.append(entire_corpus)
    return corpuses

In [289]:
def get_corpus(corpuses):
    pbar = tqdm(total=len(corpuses), ncols=1000, bar_format='{l_bar}{bar}{n_fmt}/{total_fmt}')
    pre_qas = []
    for ind, corpus in enumerate(corpuses):
        pbar.update(1)
        pbar.set_description("Extracting prepared remarks #%s" % ind)
        if corpus:
            cleaned_corpus = " ".join([para.get_text() for para in corpus]).replace("\n", " ").replace("&", "and")
            try:
                cleaned_corpus_pre_qa = cleaned_corpus[:cleaned_corpus.index("Questions and Answers")].strip()
            except:
                cleaned_corpus_pre_qa = None
            pre_qas.append(cleaned_corpus_pre_qa)
        else:
            pre_qas.append(None)
    return pre_qas

In [290]:
def get_qa(corpuses):
    pbar = tqdm(total=len(corpuses), ncols=1000, bar_format='{l_bar}{bar}{n_fmt}/{total_fmt}')
    post_qas = []
    for ind, corpus in enumerate(corpuses):
        pbar.update(1)
        pbar.set_description("Extracting Q&A #%s" % ind)
        if corpus:
            qa_reg = re.compile(".*Questions (and|&) Answers.*")
            try:
                index_qa = [ind for ind, para in enumerate(corpus) if qa_reg.search(para.get_text())][0]
                corpus_post_qa = corpus[index_qa:]
                cleaned_corpus_post_qa = [para.get_text() for para in corpus_post_qa]
                cleaned_corpus_post_qa = remove_analyst_comments(cleaned_corpus_post_qa)
                cleaned_corpus_post_qa = remove_operator_comments(cleaned_corpus_post_qa)
                cleaned_corpus_post_qa = remove_misc(cleaned_corpus_post_qa)
                key_people, cleaned_corpus_post_qa = get_key_people(cleaned_corpus_post_qa)
            except:
                cleaned_corpus_post_qa = None
        else:
            cleaned_corpus_post_qa = None
        post_qas.append([key_people, cleaned_corpus_post_qa])
    return post_qas

In [291]:
def remove_analyst_comments(cleaned_corpus_post_qa):
    analyst_present = re.compile(".*--\sAnalyst")
    cleaned_corpus_check_analyst = [analyst_present.search(para) for para in cleaned_corpus_post_qa]
    tracker_ = []
    skip_ = False
    for ind_, analyst_check in enumerate(cleaned_corpus_check_analyst):
        if skip_:
            tracker_.append(True)
            skip_ = False
            continue
        else:
            if analyst_check:
                tracker_.append(True)
                skip_ = True
            else:
                tracker_.append(False)
    corpus_no_analyst = [line_ for check_, line_ in zip(tracker_, cleaned_corpus_post_qa) if not check_]
    return corpus_no_analyst

In [292]:
def remove_operator_comments(cleaned_corpus_post_qa):
    operator_present = re.compile("Operator")
    cleaned_corpus_check_operator = [operator_present.search(para) for para in cleaned_corpus_post_qa]
    tracker_ = []
    skip_ = False
    for ind_, operator_check in enumerate(cleaned_corpus_check_operator):
        if skip_:
            tracker_.append(True)
            skip_ = False
            continue
        else:
            if operator_check:
                tracker_.append(True)
                skip_ = True
            else:
                tracker_.append(False)
    corpus_no_operator = [line_ for check_, line_ in zip(tracker_, cleaned_corpus_post_qa) if not check_]
    return corpus_no_operator

In [293]:
def remove_misc(cleaned_corpus_post_qa):
    duration_compile = re.compile(".*[D|d]uration:.*minute.*")
    call_participants_compile = re.compile(".*[C|c]all participants.*")
    more_ticker_compile = re.compile("More\s?[A-Z]{1,4}\s?analysis")
    all_earnings_compile = re.compile(".*[A|a]ll earnings call transcripts?.*")
    new_lines_compile = re.compile("(\n)+")
    tracker_ = []
    for line_ in cleaned_corpus_post_qa:
        duration_check = duration_compile.search(line_)
        call_participants_check = call_participants_compile.search(line_)
        more_ticker_check = more_ticker_compile.search(line_)
        all_earnings_check = all_earnings_compile.search(line_)
        new_lines_check = new_lines_compile.search(line_)
        if line_=="":
            tracker_.append(True)
        elif line_.lower()=='questions and answers:' or line_.lower()=='questions & answers:':
            tracker_.append(True)
        elif duration_check:
            tracker_.append(True)
        elif call_participants_check:
            tracker_.append(True)
        elif more_ticker_check:
            tracker_.append(True)
        elif all_earnings_check:
            tracker_.append(True)
        elif new_lines_check:
            tracker_.append(True)
        else:
            tracker_.append(False)
    corpus_no_misc = [line_ for check_, line_ in zip(tracker_, cleaned_corpus_post_qa) if not check_]
    return corpus_no_misc

In [294]:
def get_key_people(cleaned_corpus_post_qa):
    person_title = re.compile("\s--\s[A-Z]")
    key_people = []
    tracker_ = []
    for line_ in cleaned_corpus_post_qa:
        person_title_check = person_title.search(line_)
        if person_title_check and len(line_)<100:
            key_people.append(line_)
            tracker_.append(True)
        else:
            tracker_.append(False)
    key_people = list(set(key_people))
    corpus_key_people = [line_ for check_, line_ in zip(tracker_, cleaned_corpus_post_qa) if not check_]
    return key_people, corpus_key_people

In [295]:
corpuses = get_transcript(new_dates_transcript_list, transcripts)

In [296]:
pre_qa = get_corpus(corpuses)

In [297]:
people_post_qas = get_qa(corpuses)

In [307]:
# Combining all the information
final_list = copy.deepcopy(new_dates_transcript_list)
_ = [final_list[ind].append(time_) for ind, time_ in enumerate(announcement_times)]
_ = [final_list[ind].append(post_key_people) for ind, (post_key_people, post_) in enumerate(people_post_qas)]
_ = [final_list[ind].append(pre_) for ind, pre_ in enumerate(pre_qa)]
_ = [final_list[ind].append(post_) for ind, (post_key_people, post_) in enumerate(people_post_qas)]

In [308]:
final_df = pd.DataFrame(final_list, columns=['Ticker', 'Company', 'Fiscal_Date', 'Period_Ending', 'Transcript_Date', 'Transcript_Link',
                                             'Announcement_Time', 'Key_People', 'Prepared_Remarks', 'QandA'])
final_df = final_df.dropna()
final_df = final_df.reset_index(drop=True)
display(final_df)
final_df.info()
final_df.to_pickle("final_df.pickle")

,Ticker,Company,Fiscal_Date,Period_Ending,Transcript_Date,Transcript_Link,Announcement_Time,Key_People,Prepared_Remarks,QandA
0,NG,Novagold Resources Inc,Q2 2020,"May 31, 2020.","Jun 25, 2020",/earnings/call-transcripts/2020/06/25/novagold...,11:00 a.m.,[David Ottewell -- Vice President and Chief Fi...,"Thank you, Gillie, and good morning everyone. ...","[So, when you take into account proven and pro..."
1,SNX,SYNNEX Corporation,Q2 2020,"May 31, 2020.","Jun 25, 2020",/earnings/call-transcripts/2020/06/25/synnex-c...,5:00 p.m.,"[Marshall Witt -- Chief Financial Officer, Mar...","Thank you, Chantal, and good afternoon everyon...","[Hi Vince. Thanks for the question. Yeah, for ..."
2,PRGS,Progress Software,Q2 2020,"March 31, 2020.","Jun 25, 2020",/earnings/call-transcripts/2020/06/25/progress...,5:00 p.m.,"[Anthony Folger -- Chief Financial Officer, Yo...","Thank you, Nadia. Good afternoon, everyone, an...","[Thanks, Steve. I hope you're well, too., So S..."
3,NKE,Nike Inc,Q4 2020,"May 31, 2020.","Jun 25, 2020",/earnings/call-transcripts/2020/06/25/nike-inc...,5:00 p.m.,[John Donahoe -- President and Chief Executive...,"Thank you, operator. Hello, everyone, and than...","[Sure, Alexandra. In a funny way, I would char..."
4,CAMP,CalAmp Corp,Q1 2021,"May 31, 2020.","Jun 25, 2020",/earnings/call-transcripts/2020/06/25/calamp-c...,4:30 p.m.,[Kurt Binder -- Executive Vice President and C...,Good afternoon and welcome to CalAmp's fiscal ...,"[Thanks, Mike., Yeah, Mike. So, as you pointed..."
...,...,...,...,...,...,...,...,...,...,...
967,GTYH,GTY Technology Holdings Inc.,Q1 2020,"March 31, 2020.","May 8, 2020",/earnings/call-transcripts/2020/05/09/gty-tech...,8:30 a.m.,[Tyler Radke -- Citigroup Inc. -- Citigroup In...,"Thank you, and good morning, everyone. I'm Joh...","[Hey, Scott, thanks for joining us today. So y..."
968,EOG,EOG Resources Inc,Q1 2020,"March 31, 2020.","May 8, 2020",/earnings/call-transcripts/2020/05/09/eog-reso...,10:00 a.m.,"[Billy Helms -- Chief Operating Officer, Ezra ...","Thanks, and good morning. Thanks for joining u...","[Yes. Leo, this is Bill. And I think we're goi..."
969,PRA,ProAssurance Corp,Q1 2020,"March 31, 2020.","May 8, 2020",/earnings/call-transcripts/2020/05/09/proassur...,10:00 a.m.,[Edward Lewis Rand Jr. -- President and Chief ...,"Thank you, Jamie. In addition to our remarks o...",[I'm going to let Mike handle the bulk of that...
970,EXC,Exelon Corp,Q1 2020,"March 31, 2020.","May 8, 2020",/earnings/call-transcripts/2020/05/09/exelon-c...,10:00 a.m.,[Joseph Nigro -- Senior Executive Vice Preside...,"Thank you, Mary. Good morning, everyone, and t...","[We have seen in other jurisdictions, legislat..."


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 972 entries, 0 to 971
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Ticker             972 non-null    object
 1   Company            972 non-null    object
 2   Fiscal_Date        972 non-null    object
 3   Period_Ending      972 non-null    object
 4   Transcript_Date    972 non-null    object
 5   Transcript_Link    972 non-null    object
 6   Announcement_Time  972 non-null    object
 7   Key_People         972 non-null    object
 8   Prepared_Remarks   972 non-null    object
 9   QandA              972 non-null    object
dtypes: object(10)
memory usage: 76.1+ KB


In [309]:
final_df = pd.read_pickle("final_df.pickle")
final_df

,Ticker,Company,Fiscal_Date,Period_Ending,Transcript_Date,Transcript_Link,Announcement_Time,Key_People,Prepared_Remarks,QandA
0,NG,Novagold Resources Inc,Q2 2020,"May 31, 2020.","Jun 25, 2020",/earnings/call-transcripts/2020/06/25/novagold...,11:00 a.m.,[David Ottewell -- Vice President and Chief Fi...,"Thank you, Gillie, and good morning everyone. ...","[So, when you take into account proven and pro..."
1,SNX,SYNNEX Corporation,Q2 2020,"May 31, 2020.","Jun 25, 2020",/earnings/call-transcripts/2020/06/25/synnex-c...,5:00 p.m.,"[Marshall Witt -- Chief Financial Officer, Mar...","Thank you, Chantal, and good afternoon everyon...","[Hi Vince. Thanks for the question. Yeah, for ..."
2,PRGS,Progress Software,Q2 2020,"March 31, 2020.","Jun 25, 2020",/earnings/call-transcripts/2020/06/25/progress...,5:00 p.m.,"[Anthony Folger -- Chief Financial Officer, Yo...","Thank you, Nadia. Good afternoon, everyone, an...","[Thanks, Steve. I hope you're well, too., So S..."
3,NKE,Nike Inc,Q4 2020,"May 31, 2020.","Jun 25, 2020",/earnings/call-transcripts/2020/06/25/nike-inc...,5:00 p.m.,[John Donahoe -- President and Chief Executive...,"Thank you, operator. Hello, everyone, and than...","[Sure, Alexandra. In a funny way, I would char..."
4,CAMP,CalAmp Corp,Q1 2021,"May 31, 2020.","Jun 25, 2020",/earnings/call-transcripts/2020/06/25/calamp-c...,4:30 p.m.,[Kurt Binder -- Executive Vice President and C...,Good afternoon and welcome to CalAmp's fiscal ...,"[Thanks, Mike., Yeah, Mike. So, as you pointed..."
...,...,...,...,...,...,...,...,...,...,...
967,GTYH,GTY Technology Holdings Inc.,Q1 2020,"March 31, 2020.","May 8, 2020",/earnings/call-transcripts/2020/05/09/gty-tech...,8:30 a.m.,[Tyler Radke -- Citigroup Inc. -- Citigroup In...,"Thank you, and good morning, everyone. I'm Joh...","[Hey, Scott, thanks for joining us today. So y..."
968,EOG,EOG Resources Inc,Q1 2020,"March 31, 2020.","May 8, 2020",/earnings/call-transcripts/2020/05/09/eog-reso...,10:00 a.m.,"[Billy Helms -- Chief Operating Officer, Ezra ...","Thanks, and good morning. Thanks for joining u...","[Yes. Leo, this is Bill. And I think we're goi..."
969,PRA,ProAssurance Corp,Q1 2020,"March 31, 2020.","May 8, 2020",/earnings/call-transcripts/2020/05/09/proassur...,10:00 a.m.,[Edward Lewis Rand Jr. -- President and Chief ...,"Thank you, Jamie. In addition to our remarks o...",[I'm going to let Mike handle the bulk of that...
970,EXC,Exelon Corp,Q1 2020,"March 31, 2020.","May 8, 2020",/earnings/call-transcripts/2020/05/09/exelon-c...,10:00 a.m.,[Joseph Nigro -- Senior Executive Vice Preside...,"Thank you, Mary. Good morning, everyone, and t...","[We have seen in other jurisdictions, legislat..."
